In [ ]:
import talib as ta

from datetime import datetime
import pandas as pd
import numpy as np
import pickle

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

def get_score(model, x_train=0, x_test=0):
    if not x_train or not x_test:
        x_train = X_train
        x_test = X_test
    y_pred = model.predict(x_test)
    print('train: {}'.format(model.score(x_train, y_train) * 100))
    print('test: {}'.format(model.score(x_test, y_test) * 100))
    print('accuracy score: {}'.format(accuracy_score(y_test, y_pred) * 100))

params_l1 = {
    'log__penalty': ['l1'],
    'log__C': np.logspace(-4, 4, 20),
    'log__tol': [0.00001, 0.0001, 0.001, 0.01, 0.10],
    'log__class_weight': [None, 'balanced'],
    'log__solver': ['saga'],
    'log__multi_class': ['ovr', 'multinomial', 'auto']
}
params_l2 = {
    'penalty': ['l2'],
    'C': np.logspace(-4, 4, 20),
    'tol': [0.00001, 0.0001, 0.001, 0.01, 0.10],
    'class_weight': [None, 'balanced'],
    'solver': ['newton-cg', 'lbfgs', 'sag'],
    'multi_class': ['ovr', 'multinomial', 'auto']
}
best_params = knn_cv.best_params_

In [2]:
df = pd.read_csv('data/data3_H1.csv', parse_dates=['Datetime'], index_col='Datetime')

In [3]:
X = df.drop('Target', axis=1).values
y = df.Target.values

X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.30, random_state=42)

half_split = int(len(X_tmp) / 2)
X_test = X_tmp[:half_split]
X_final = X_tmp[half_split:]
y_test = y_tmp[:half_split]
y_final = y_tmp[half_split:]

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_final = scaler.transform(X_final)

In [5]:
poly = PolynomialFeatures()
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)
X_final = poly.transform(X_final)

In [6]:
X_train.shape

(67713, 351)

In [8]:
X_test.shape

(14510, 351)

In [9]:
log = LogisticRegression()
log.fit(X_train, y_train)

/anaconda2/envs/python3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [ ]:
get_score(log)

train: 53.864102904907476
test: 52.370778773259815
accuracy score: 52.370778773259815


In [ ]:
log_cv2 = GridSearchCV(LogisticRegression(), params_l2, cv=3, verbose=True, n_jobs=-1)
log_cv2.fit(X_train, y_train)
print('best score: {}'.format(log_cv2.best_score_ * 100))

Fitting 3 folds for each of 1800 candidates, totalling 5400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.3min
/anaconda2/envs/python3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 41.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 62.6min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 167.7min


In [ ]:
ada_clf = AdaBoostClassifier(
    LogisticRegression(**best_params),
    n_estimators=1000,
    algorithm="SAMME.R",
    learning_rate=0.01
)
ada_clf.fit(X_train, y_train)
get_score(ads_clf)

In [ ]:
bag_log = BaggingClassifier(
    LogisticRegression(**best_params),
    n_estimators=1000,
    max_samples=500,
    bootstrap=True
)
bag_log.fit(X_train, y_train)
get_score(bag_log)